In [ ]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [ ]:
y = x + 2
print(y)
print(x.backward)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
<bound method Tensor.backward of tensor([[1., 1.],
        [1., 1.]], requires_grad=True)>


In [ ]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
import numpy as np

In [ ]:
# 현재 실습하고 있는 파이썬 코드를 재실행해도 다음에도 같은 결과가 나오도록 랜덤 시드(random seed)를 줍니다.
torch.manual_seed(0)

# Auto grad

Tensor
패키지의 중심에는 torch.Tensor 클래스가 있습니다. 만약 .requires_grad 속성을 True 로 설정하면, 그 tensor에서 이뤄진 모든 연산들을 추적(track)하기 시작합니다. 계산이 완료된 후 .backward() 를 호출하여 모든 변화도(gradient)를 자동으로 계산할 수 있습니다. 이 Tensor의 변화도는 .grad 속성에 누적됩니다.

Tensor가 기록을 추적하는 것을 중단하게 하려면, .detach() 를 호출하여 연산 기록으로부터 분리(detach)하여 이후 연산들이 추적되는 것을 방지할 수 있습니다.

기록을 추적하는 것(과 메모리를 사용하는 것)을 방지하기 위해, 코드 블럭을 with torch.no_grad(): 로 감쌀 수 있습니다. 이는 특히 변화도(gradient)는 필요없지만, requires_grad=True 가 설정되어 학습 가능한 매개변수를 갖는 모델을 평가(evaluate)할 때 유용합니다.

Autograd 구현에서 매우 중요한 클래스가 하나 더 있는데, 이것은 바로 Function 클래스입니다.

Tensor 와 Function 은 서로 연결되어 있으며, 모든 연산 과정을 부호화(encode)하여 순환하지 않는 그래프(acyclic graph)를 생성합니다. 각 tensor는 .grad_fn 속성을 갖고 있는데, 이는 Tensor 를 생성한 Function 을 참조하고 있습니다. (단, 사용자가 만든 Tensor는 예외로, 이 때 grad_fn 은 None 입니다.)

도함수를 계산하기 위해서는 Tensor 의 .backward() 를 호출하면 됩니다. 만약 Tensor 가 스칼라(scalar)인 경우(예. 하나의 요소 값만 갖는 등)에는 backward 에 인자를 정해줄 필요가 없습니다. 하지만 여러 개의 요소를 갖고 있을 때는 tensor의 모양을 gradient 의 인자로 지정할 필요가 있습니다.

In [4]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [18]:
y = x + 2
print(y)
print(x.backward)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
<bound method Tensor.backward of tensor([[1., 1.],
        [1., 1.]], requires_grad=True)>


In [6]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [7]:
a = torch.randn(2, 2)
print(a)
print(a*3)
print(a-1)
a = ((a * 3) / (a - 1))
print(a) #elementwise

tensor([[ 1.5410, -0.2934],
        [-2.1788,  0.5684]])
tensor([[ 4.6230, -0.8803],
        [-6.5364,  1.7053]])
tensor([[ 0.5410, -1.2934],
        [-3.1788, -0.4316]])
tensor([[ 8.5453,  0.6806],
        [ 2.0562, -3.9514]])


In [11]:
print(4.6230/0.5410)
print(-0.8803/-1.2934)

8.5452865064695
0.6806092469460336


In [19]:
print(a.requires_grad) # 기본값이 False 기때문에
a.requires_grad_(True) #a에 저장하게 함
print(a.requires_grad)
b = (a * a).sum()
print(b)
print(b.grad_fn)

True
True
tensor(93.3274, grad_fn=<SumBackward0>)


Gradients
---------
Let's backprop now
Because ``out`` contains a single scalar, ``out.backward()`` is
equivalent to ``out.backward(torch.tensor(1))``.



In [24]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [25]:
y = x + 2
print(y)
print(x.backward)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
<bound method Tensor.backward of tensor([[1., 1.],
        [1., 1.]], requires_grad=True)>


In [26]:
z = y * y * 3
out = z.mean() 

print(z, out)



tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


print gradients d(out)/dx




In [27]:
out.backward() # 첫 변수까지의 

In [29]:
print(x.grad)
print(y.grad)
print(z.grad)
#z=3(x+2)^2=3x^2+12x+12
#dz/dx = 6x+12
#dout/dx = dout/dz *dz/dy *dy/dx
#dout/dx의 size = (2,2) 가 될거야 
#dout/dx1= dout/dz1 *dz1/dy1 *dy1/dx1
        #= 1/4 * 6y1 * 1
        #= 1/4 * 6*3 * 1
        # 4.5 


tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])
None
None


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  This is separate from the ipykernel package

You should have got a matrix of ``4.5``. Let’s call the ``out``
*Tensor* “$o$”.
We have that $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.
Therefore,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, hence
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.



You can do many crazy things with autograd!



In [85]:
torch.manual_seed(0)
x = torch.randn(3, requires_grad=True)
print('------------------x-------------------')
print(f'x=\n{x}')

y = x * 2

print('------------------y-------------------')
print(f'y=\n{y}')
print(y.data.norm()) # 벡터의 크기 sqrt(sum(squared))
print(4*x.data.norm()) # y=2x ㅣyㅣ=sqrt(4x^2)=2ㅣxㅣ
while y.data.norm() < 1000:
    y = y * 2
print('-------after y norm--------------')
print(f'y=\n{y}')


------------------x-------------------
x=
tensor([ 1.5410, -0.2934, -2.1788], requires_grad=True)
------------------y-------------------
y=
tensor([ 3.0820, -0.5869, -4.3576], grad_fn=<MulBackward0>)
tensor(5.3695)
tensor(10.7390)
-------after y norm--------------
y=
tensor([  788.9900,  -150.2356, -1115.5402], grad_fn=<MulBackward0>)


In [86]:
original=torch.tensor([1,1,1],dtype=torch.long)
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
# x에 대한 y의 미분값에 gradients 내 원소들이 
# elementwise로 곱해짐
# y.backward() 이건 y가 scalar 값일 때 가능
# jacobian 형태를 유지하려면 저렇게 [1,1,1]을 y.backward([1,1,1]) 해줘야함
y.backward(original)
print(f'x.grad=\n{x.grad}')


x.grad=
tensor([512., 512., 512.])


In [87]:
torch.manual_seed(0)
x = torch.randn(3, requires_grad=True)
print('------------------x-------------------')
print(f'x=\n{x}')

y = x * 2

print('------------------y-------------------')
print(f'y=\n{y}')
print(y.data.norm()) # 벡터의 크기 sqrt(sum(squared))
print(4*x.data.norm()) # y=2x ㅣyㅣ=sqrt(4x^2)=2ㅣxㅣ
while y.data.norm() < 1000:
    y = y * 2
print('-------after y norm--------------')
print(f'y=\n{y}')

y.backward(gradients)
print(f'x.grad=\n{x.grad}')

------------------x-------------------
x=
tensor([ 1.5410, -0.2934, -2.1788], requires_grad=True)
------------------y-------------------
y=
tensor([ 3.0820, -0.5869, -4.3576], grad_fn=<MulBackward0>)
tensor(5.3695)
tensor(10.7390)
-------after y norm--------------
y=
tensor([  788.9900,  -150.2356, -1115.5402], grad_fn=<MulBackward0>)
x.grad=
tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


You can also stop autograd from tracking history on Tensors
with ``.requires_grad=True`` by wrapping the code block in
``with torch.no_grad()``:



In [52]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)

True
True
False


**Read Later:**

Documentation of ``autograd`` and ``Function`` is at
http://pytorch.org/docs/autograd



In [69]:
from torch.autograd import Variable
a=torch.rand(5)
a=Variable(a,requires_grad=True)
b=a+2
b.requires_grad_(True)

c=b**(1/2)
c.requires_grad_(True) 

out=c.sum()
out.requires_grad_(True)

#b.backward() :grad can be implicitly created 
              #only for scalar outputs
#중간 과정의 미분값은 requires_grad_(True)를 해줘야
             #할수 있음
out.backward() #out을 가지고 a까지의 미분값을 구한다는 거야

print('--------------a----------------')
print(a.data)
print(a.grad)
print(a.grad_fn)

print('--------------b----------------')
print(b)
print(b.data)
print(b.grad)
print(b.grad_fn)

print('--------------c----------------')
print(c)
print(c.data)
print(c.grad)
print(c.grad_fn)

print('--------------out----------------')
print(out)
print(out.data)
print(out.grad)
print(out.grad_fn)

--------------a----------------
tensor([0.1143, 0.4725, 0.5751, 0.2952, 0.7967])
tensor([0.3439, 0.3180, 0.3116, 0.3300, 0.2990])
None
--------------b----------------
tensor([2.1143, 2.4725, 2.5751, 2.2952, 2.7967], grad_fn=<AddBackward0>)
tensor([2.1143, 2.4725, 2.5751, 2.2952, 2.7967])
None
--------------c----------------
tensor([1.4541, 1.5724, 1.6047, 1.5150, 1.6723], grad_fn=<PowBackward0>)
tensor([1.4541, 1.5724, 1.6047, 1.5150, 1.6723])
None
--------------out----------------
tensor(7.8185, grad_fn=<SumBackward0>)
tensor(7.8185)
None


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
/usr/local/lib/python3.6/dist-packages/ipykern